In [13]:
import pandas as pd
import category_encoders as ce
import numpy as np

In [72]:
from mlxtend.classifier import StackingCVClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import catboost as cb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [38]:
features = pd.read_csv('features_13.csv').set_index('person')

In [39]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')
labels = pd.read_csv('labels_training_set.csv')

In [41]:
training = labels.set_index('person').join(features)
train_labels = training['label']
train_data = training.drop('label',axis=1)

In [43]:
predict_data = kaggle.set_index('person').join(features)

## Stacking

In [47]:
cfeats = []
for i,v in enumerate(features.columns):
    if features[v].dtype.name == 'object':
        cfeats.append(i)

In [48]:
cb_classifier = cb.CatBoostClassifier(iterations=400,
                                 depth=8,
                                 l2_leaf_reg=50,
                                 random_strength=0.1,
                                 bagging_temperature=0,
                                 border_count=128,
                                 cat_features=cfeats)

xg_params = {'colsample_bytree': 0.7476502755257962,
 'gamma': 1,
 'learning_rate': 0.015344184565596445,
 'max_delta_step': 0,
 'max_depth': 11,
 'min_child_weight': 11,
 'n_estimators': 324,
 'subsample': 0.7718583635196341}
xg_classifier = xgb.XGBClassifier(**xg_params)

knn_classifier = KNeighborsClassifier(p=2,n_neighbors=10)

rf_classsifier = RandomForestClassifier(n_estimators=100)

In [75]:
logreg_classifier = LogisticRegression()

In [50]:
p_xgb = Pipeline(steps=[('encoding',ce.TargetEncoder()),('fit',xg_classifier)])

In [51]:
p_knn = Pipeline(steps=[('encoding',ce.TargetEncoder()),('normalization',StandardScaler()),('fit',knn_classifier)])

In [52]:
p_rf = Pipeline(steps=[('encoding',ce.TargetEncoder()),('fit',rf_classsifier)])

In [76]:
stack = StackingCVClassifier(classifiers=[p_xgb,cb_classifier,p_knn,p_rf],meta_classifier=logreg_classifier,use_probas=True)

In [77]:
stack.fit(np.array(train_data.fillna(0)),np.array(train_labels))

0:	learn: 0.6496909	total: 43.6ms	remaining: 17.4s
1:	learn: 0.6094836	total: 161ms	remaining: 32.1s
2:	learn: 0.5728921	total: 191ms	remaining: 25.3s
3:	learn: 0.5397279	total: 209ms	remaining: 20.7s
4:	learn: 0.5094303	total: 227ms	remaining: 17.9s
5:	learn: 0.4806460	total: 348ms	remaining: 22.9s
6:	learn: 0.4572938	total: 380ms	remaining: 21.3s
7:	learn: 0.4332332	total: 493ms	remaining: 24.2s
8:	learn: 0.4135335	total: 610ms	remaining: 26.5s
9:	learn: 0.3960945	total: 739ms	remaining: 28.8s
10:	learn: 0.3801854	total: 853ms	remaining: 30.2s
11:	learn: 0.3625603	total: 879ms	remaining: 28.4s
12:	learn: 0.3463589	total: 997ms	remaining: 29.7s
13:	learn: 0.3332094	total: 1.11s	remaining: 30.6s
14:	learn: 0.3219644	total: 1.22s	remaining: 31.4s
15:	learn: 0.3086946	total: 1.35s	remaining: 32.4s
16:	learn: 0.2985275	total: 1.47s	remaining: 33.2s
17:	learn: 0.2870554	total: 1.5s	remaining: 31.9s
18:	learn: 0.2795252	total: 1.63s	remaining: 32.7s
19:	learn: 0.2724891	total: 1.74s	remaini

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\pipeline.py:381: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with

0:	learn: 0.6491822	total: 160ms	remaining: 1m 3s
1:	learn: 0.6067448	total: 309ms	remaining: 1m 1s
2:	learn: 0.5701507	total: 354ms	remaining: 46.9s
3:	learn: 0.5334181	total: 516ms	remaining: 51.1s
4:	learn: 0.5014920	total: 615ms	remaining: 48.6s
5:	learn: 0.4717177	total: 770ms	remaining: 50.6s
6:	learn: 0.4491349	total: 941ms	remaining: 52.8s
7:	learn: 0.4253387	total: 1.04s	remaining: 51.1s
8:	learn: 0.4025223	total: 1.2s	remaining: 52.1s
9:	learn: 0.3858100	total: 1.25s	remaining: 49s
10:	learn: 0.3667167	total: 1.41s	remaining: 49.7s
11:	learn: 0.3525849	total: 1.55s	remaining: 50.3s
12:	learn: 0.3358909	total: 1.71s	remaining: 50.8s
13:	learn: 0.3245482	total: 1.73s	remaining: 47.7s
14:	learn: 0.3092934	total: 1.89s	remaining: 48.4s
15:	learn: 0.2969067	total: 2.05s	remaining: 49.2s
16:	learn: 0.2882999	total: 2.2s	remaining: 49.6s
17:	learn: 0.2773823	total: 2.36s	remaining: 50.1s
18:	learn: 0.2676928	total: 2.53s	remaining: 50.8s
19:	learn: 0.2606761	total: 2.69s	remaining: 

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


StackingCVClassifier(classifiers=[Pipeline(memory=None,
     steps=[('encoding', TargetEncoder(cols=None, drop_invariant=False, handle_unknown='impute',
       impute_missing=True, min_samples_leaf=1, return_df=True,
       smoothing=1.0, verbose=0)), ('fit', XGBClassifier(base_score=0.5, booster='gbtree', colsample_byl...s=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])],
           cv=2,
           meta_classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
           shuffle=True, store_train_meta_features=False, stratify=True,
           use_clones=True, use_features_in_secondary=False,
           use_probas=True, verbose=0)

In [78]:
prediction = stack.predict_proba(np.array(predict_data.fillna(0)))[:,1]

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\pipeline.py:381: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [79]:
prediction.mean()

0.0517995340480866

In [81]:
submit = kaggle.set_index('person')
submit['label'] = prediction
submit.to_csv('submit.csv')